In [13]:
import keras.callbacks
import tensorflow as tf

from keras.models import Model
from keras.layers import UpSampling2D, ZeroPadding2D, Conv2DTranspose, Input, concatenate
from keras.layers.core import Activation 
from keras.layers.convolutional import Conv2D, MaxPooling2D, Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.preprocessing import image
from keras import backend as K
from keras.callbacks import TensorBoard, EarlyStopping
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.engine.topology import get_source_inputs

from sklearn.model_selection import train_test_split

import os 
import sys
import numpy as np 

import matplotlib.pyplot as plt

import warnings

In [14]:
print(sys.version)
print(keras.__version__)
#print(tf.__version__)

3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]
2.2.4


# Load & preproc data

In [15]:
#!unzip all.zip

In [16]:
path = r'C:\Users\RZD.GENERATION\Downloads\data\all' 
pathX = path + r'\train'
pathY = path + r'\trainmask'

#path = '/content/all'
#pathX = path + '/train'
#pathY = path + '/trainmask'
 
data_names = os.listdir(pathX)
dataX = []
dataY = []

h = 256
w = 256
 
for count, smpl in enumerate(data_names): #some preprac can be done here
    img = image.img_to_array(image.load_img(pathX+"/"+smpl))  #.resize((h,w)))
    mask = image.img_to_array(image.load_img(pathY+"/"+smpl))  #.resize((h,w)))
    if img.shape[0] not in [360, h]:
        img = img[8:,:,:]
        mask = mask[8:,:,:]
    dataX.append(img)
    dataY.append(mask)
    if count == 1000:
        break

dataX = np.array(dataX)/255.
dataY = (np.array(dataY)[:,:,:,0])/255. 
dataY = np.expand_dims(dataY, axis=3)

(x_train, x_test, y_train, y_test) = train_test_split(dataX, dataY, test_size=0.25)

## Augmentation

In [ ]:
#flip
for i in range(len(dataX)):
    dataX.append(tf.image.flip_left_right(dataX[i]))
    dataY.append(tf.image.flip_left_right(dataY[i]))
    

#change brightness
for i in range(len(dataX)):
    dataX.append(tf.image.random_brightness(dataX[i], 0.05))
    dataY.append(dataY[i])

In [17]:
#dataX = np.array(dataX)/255.
#dataY = (np.array(dataY)[:,:,:,0])/255. 
#dataY = np.expand_dims(dataY, axis=3)

#(x_train, x_test, y_train, y_test) = train_test_split(dataX, dataY, test_size=0.25)

In [18]:
img_rows, img_cols = x_train.shape[1], x_train.shape[2] 
channels = x_train.shape[3]

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], channels, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], channels, img_rows, img_cols)
    input_shape = (channels, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, channels)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, channels)
    input_shape = (img_rows, img_cols, channels)

In [19]:
print('train:')
print(x_train.shape)
print(y_train.shape)
print('test:')
print(x_test.shape)
print(y_test.shape)

train:
(750, 360, 480, 3)
(750, 360, 480, 1)
test:
(251, 360, 480, 3)
(251, 360, 480, 1)


In [20]:
def viz_res(model, n=10):
    y_test1 = np.array([i[:,:,0] for i in y_test])

    segm_imgs = model.predict(x_test[:n])
    #for sigmoid activ
    segm_imgs[segm_imgs < 0.5] = 0.
    segm_imgs[segm_imgs >= 0.5] = 1.
    segm_imgs = np.array([i[:,:,0] for i in segm_imgs])

    for i in range(n):
        fig, axarr = plt.subplots(1, 3, figsize=(30, 30))
        axarr[0].imshow(x_test[i])
        axarr[1].imshow(y_test1[i], cmap='gray')
        axarr[2].imshow(segm_imgs[i], cmap='gray') 
        for j in range(3):
            axarr[j].get_xaxis().set_visible(False)
            axarr[j].get_yaxis().set_visible(False)
        plt.show()

# Loss & metrics

In [21]:
def iou(y_true, y_pred, smooth=1):
    y_pred = K.round(y_pred) ###
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum( y_pred_f) - intersection
    iou = (intersection + smooth) / (union + smooth)
    return iou

In [22]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    
    tp = K.sum(y_true_f*y_pred_f)
    fp = K.sum((1-y_true_f)*y_pred_f)
    fn = K.sum(y_true_f*(1-y_pred_f))

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    return 2*p*r / (p+r+K.epsilon())

In [23]:
def dice_coef(y_true, y_pred, smooth=1):
    y_pred = K.round(y_pred)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_coef_loss(y_true, y_pred)

In [24]:
def kulch_coef(y_true, y_pred, smooth=1):
    y_pred = K.round(y_pred)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - 2*intersection + smooth)

def kulch_coef_loss(y_true, y_pred):
    return 1-kulch_coef(y_true, y_pred)

def bce_kulch_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + kulch_coef_loss(y_true, y_pred)

In [25]:
earlyStopping = EarlyStopping(monitor='val_loss', 
                              patience=2, #number of epochs with no improvement after which training will be stopped
                              verbose=1, #training will stop when the quantity monitored has stopped decreasing
                              min_delta = 0.0001,
                              mode='min')

#modelCheckpoint = ModelCheckpoint(checkpoint_loc,
#                                  monitor = 'val_loss', 
#                                  save_best_only = True, 
#                                  mode = 'min', 
#                                  verbose = 1,
#                                  save_weights_only = True)

tensorboard = TensorBoard(log_dir=r'C:\Users\RZD.GENERATION\Downloads\keras_pract\caffenet_log', histogram_freq=0, write_graph=True, write_images=True)

callbacks_list = [earlyStopping, tensorboard]

# SqueezeNet for segmentation

In [26]:
#modified from keras squeezenet pack

sq1x1 = 'squeeze1x1'
exp1x1 = 'expand1x1'
exp3x3 = 'expand3x3'
relu = 'relu_'

# Modular function for Fire Node

def fire_module(x, fire_id, squeeze=16, expand=64, add=''):
    s_id = 'fire' + str(fire_id) + '/'

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3
    
    x = Convolution2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1 + add)(x)
    x = Activation('relu', name=s_id + relu + sq1x1 + add)(x)

    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + exp1x1 + add)(x)
    left = Activation('relu', name=s_id + relu + exp1x1 + add)(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3 + add)(x)
    right = Activation('relu', name=s_id + relu + exp3x3 + add)(right)

    x = concatenate([left, right], axis=channel_axis, name=s_id + 'concat' + add)
    return x


# Original SqueezeNet from paper.

def SqueezeNet(input_tensor=None, 
               input_shape=None,
              dec='my'): #dec from ['my', 'FCN8', 'FCN32']
   
    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
            
    #encoder

    x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
    x = Activation('relu', name='relu_conv1')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    #decoder
    
    if dec == 'my':
        x = fire_module(x, fire_id=9, squeeze=64, expand=256, add='dec')
        x = fire_module(x, fire_id=8, squeeze=64, expand=256, add='dec')
        x = fire_module(x, fire_id=7, squeeze=48, expand=192, add='dec')
        x = fire_module(x, fire_id=6, squeeze=48, expand=192, add='dec')
    
        x = UpSampling2D(name='upsample1')(x)
        x = ZeroPadding2D((1,0))(x) 
    
        x = fire_module(x, fire_id=5, squeeze=32, expand=128, add='dec')
        x = fire_module(x, fire_id=4, squeeze=32, expand=128, add='dec')
    
        x = UpSampling2D(name='upsample2')(x) 
        x = ZeroPadding2D((0,1))(x)
    
        x = fire_module(x, fire_id=3, squeeze=16, expand=64, add='dec')
        x = fire_module(x, fire_id=2, squeeze=16, expand=64, add='dec')
    
        x = UpSampling2D(name='upsample3')(x)
        x = ZeroPadding2D((2,2))(x)
    
        x = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', name='conv_prefinal')(x) #not valid here
        
    if dec == 'FCN32': #needed to be resized 198x198
        x = Conv2DTranspose(1, (3, 3), strides=(18, 18), padding='same', name='conv_prefinal')(x)
        
    #if dec == 'FCN8': #needed to be resized
        #x = Conv2DTranspose(pool2.get_shape().as_list()[-1], (0, 2), strides=(2, 2), padding='valid', name='transpose1')(x)
        #x = Add()([x, pool2])
        #x = Conv2DTranspose(pool1.get_shape().as_list()[-1], (2, 1), strides=(2, 2), padding='same', name='transpose2')(x)
        #x = Add()([x, pool1])
        #x = ZeroPadding2D((1,1))(x)
        #x = Conv2DTranspose(64, (16, 16), strides=(8, 8), padding='same', name='transpose3')(x)
     
    model = Conv2D(1, (3, 3), padding='same', activation='sigmoid', name='conv_final')(x)     

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    model = Model(inputs, model, name='squeezenet')
    
    model.compile(loss=bce_kulch_loss, optimizer=Adam(lr=1e-4), metrics=[f1, iou])

    return model

In [28]:
warnings.filterwarnings('ignore')

batch_size = 64
epochs = 5

model = SqueezeNet(input_shape=(360, 480, 3))

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 360, 480, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 179, 239, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 179, 239, 64) 0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 89, 119, 64)  0           relu_conv1[0][0]                 
__________________________________________________________________________________________________
fire2/sque

In [29]:
warnings.filterwarnings('ignore')

batch_size = 32
epochs = 5

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          shuffle=True,
          #callbacks = callbacks_list,
          validation_data=(x_test, y_test))

W0723 12:22:20.314453  2256 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 750 samples, validate on 251 samples
Epoch 1/5
160/750 [=====>........................] - ETA: 14:47 - loss: 1.6885 - f1: 0.0000e+00 - iou: 0.0012

KeyboardInterrupt: 